In [ ]:
input_geo_hash = '9p'

In [ ]:
import geohash
import datetime

def parseLine(line):
    variables = line.split("\t")
    try:
        milliseconds = int(variables[0])
        dt = datetime.datetime.fromtimestamp(milliseconds/1000.0)

        lat = float(variables[1])
        lon = float(variables[2])
        temperature = float(variables[10])
        precipitation = float(variables[13])
        gh = geohash.encode(lat, lon)
        return (gh, dt.month, temperature, precipitation)
    except:
        return ('', 0, 0, 0)

text_file = sc.textFile("hdfs://orion11:12001/pj3/3hr/2015/")

# (GeoHash, month, temperature, precipitation)
parsed_data = text_file \
    .map(lambda line: parseLine(line)) \
    .filter(lambda data: data[0].startswith(input_geo_hash)) \


In [ ]:
# average temperature
month_temperature_pair = parsed_data.map(lambda data: (data[1], data[2]))

allTemps = month_temperature_pair.groupByKey()

avg_temp_dict = {}
for element in allTemps.collect():
    count = 0
    temp_sum = 0
    for temp in element[1]:
        count += 1
        temp_sum += temp
    avg_temp_dict["{:02d}".format(element[0])] = temp_sum/count
    print((element[0], temp_sum/count))

In [ ]:
# maximum temperature

max_temp_dict = {}
maxTemps = month_temperature_pair.reduceByKey(lambda a, b: max(a,b))
for element in maxTemps.collect():
    print(element)
    max_temp_dict["{:02d}".format(element[0])] = element[1]


In [ ]:
# minimum teperature
min_temp_dict = {}
minTemps = month_temperature_pair.reduceByKey(lambda a, b: min(a,b))
for element in minTemps.collect():
    print(element)
    min_temp_dict["{:02d}".format(element[0])] = element[1]

In [ ]:
# average precipitation
avg_prec_dict = {}
month_precipitation_pair = parsed_data.map(lambda data: (data[1], data[3]))
month_precipitation_grouped = month_precipitation_pair.groupByKey();
for mp in month_precipitation_grouped.collect():
    count = 0;
    rain_sum = 0;
    for rain in mp[1]:
        count += 1
        rain_sum += rain
    print((mp[0], rain_sum/count))
    avg_prec_dict["{:02d}".format(mp[0])] = rain_sum/count